In [1]:
from __future__ import division

import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import butter

from wyrm import processing as proc
from wyrm import plot
from wyrm import io

ModuleNotFoundError: No module named 'wyrm'

In [2]:
cnt = io.load_brain_vision_data('data/mi/VPkg_08_08_07/imag_arrowVPkg.vhdr')

In [3]:
# remove unneeded channels
#cnt = proc.remove_channels(cnt, ['EMG.*', 'EOG.*', 'FT.*', 'TP.*', 'I.*', 'P9', 'P10', 'OI.*'])
cnt = proc.remove_channels(cnt, ['EMG.*', 'EOG.*'])

In [4]:
# band pass data
fn = cnt.fs / 2
b, a = butter(4, [10 / fn, 14 / fn], btype='band')
cnt = proc.lfilter(cnt, b, a)

In [5]:
# subsampling
cnt = proc.subsample(cnt, 100)

In [6]:
# for mi we use ~750-3500ms range
mrk_def = {'class 1': ['S  1'],
           'class 2': ['S  2'],
           'class 3': ['S  3']
           }
epo = proc.segment_dat(cnt, mrk_def, [-100, 3500])
epo = proc.correct_for_baseline(epo, [-100, 0])

In [7]:
# rectify channels
epo2 = proc.rectify_channels(epo)

In [8]:
epo_avg = proc.calculate_classwise_average(epo2)
for i, e in enumerate(epo_avg.class_names):
    plot.plot_channels(proc.select_epochs(epo_avg, [i]))
plt.show()

In [9]:
epo = proc.select_ival(epo, [750, 3500])

In [10]:
data_left = proc.select_classes(epo, [0]).data
data_right = proc.select_classes(epo, [1]).data
w, a, d = proc.calculate_csp(data_left, data_right)
# the interesting channels are usuall c3, c4 and cz
for ii, i in enumerate([0, 1, 2, -3, -2, -1]):
    plt.subplot(2, 3, ii+1)
    plot.plot_scalp(a[:,i], epo.axes[-1])
    plt.title(i)
plt.show()